In [1]:
!pip install pyspark
from pyspark.sql import SparkSession

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 54 kB/s 
     |████████████████████████████████| 199 kB 68.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=135798a50f5349872641414812c35f6d8b552ce0c5c70946fcd0b1186fee6ace
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Level1 DataFrame API (PySpark)

In [3]:
file = 'gdrive/MyDrive/wc2018-players.csv'

In [4]:
spark=SparkSession.builder.appName('sparkSQL').getOrCreate()

In [5]:
data = spark.read.csv(file, header=True, inferSchema=True)

In [6]:
data.show(10)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
|Argentina|  4|  DF|  ANSALDI Cristian|20.09.1986|   ANSALDI|     Torino FC (ITA)|   181|    73|
|Argentina|  5|  MF|      BIGLIA Lucas|30.01.1986|    BIGLIA|      AC Milan (ITA)|   175|    73|
|Argentina|  7|  MF|       BAN

In [7]:
data.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



In [8]:
#1. Show names & height of players by adding 1 to the height column

data.select('FIFA Popular Name', data['Height'] + 1).show(5)

+------------------+------------+
| FIFA Popular Name|(Height + 1)|
+------------------+------------+
|TAGLIAFICO Nicolas|         170|
|    PAVON Cristian|         170|
|    LANZINI Manuel|         168|
|    SALVIO Eduardo|         168|
|      MESSI Lionel|         171|
+------------------+------------+
only showing top 5 rows



In [9]:
#2. Show player name & return True for players (height > 170) and False for players (height < 170)

from pyspark.sql.functions import udf

def helper(a):
  if a > 170:
    return True
  else:
    return False

helper_df = udf(helper)

newdf = data.withColumn('Height > 170?',helper_df(data['Height']))
newdf.select('FIFA Popular Name', 'Height > 170?').show()

+------------------+-------------+
| FIFA Popular Name|Height > 170?|
+------------------+-------------+
|TAGLIAFICO Nicolas|        false|
|    PAVON Cristian|        false|
|    LANZINI Manuel|        false|
|    SALVIO Eduardo|        false|
|      MESSI Lionel|        false|
|  ANSALDI Cristian|         true|
|      BIGLIA Lucas|         true|
|       BANEGA Ever|         true|
| MASCHERANO Javier|         true|
|      DYBALA Paulo|         true|
|     AGUERO Sergio|         true|
|   HIGUAIN Gonzalo|         true|
|    DI MARIA Angel|         true|
|  LO CELSO Giovani|         true|
|  MEZA Maximiliano|         true|
|      ACUNA Marcos|         true|
|CABALLERO Wilfredo|         true|
|   MERCADO Gabriel|         true|
|  OTAMENDI Nicolas|         true|
|       ROJO Marcos|         true|
+------------------+-------------+
only showing top 20 rows



In [10]:
#3. Same as above but asking for 0/1 not true/false

def helper(a):
  if a > 170:
    return 1
  else:
    return 0

helper_df = udf(helper)

newdf = data.withColumn('Height > 170?',helper_df(data['Height']))
newdf.select('FIFA Popular Name', 'Height > 170?').show()

+------------------+-------------+
| FIFA Popular Name|Height > 170?|
+------------------+-------------+
|TAGLIAFICO Nicolas|            0|
|    PAVON Cristian|            0|
|    LANZINI Manuel|            0|
|    SALVIO Eduardo|            0|
|      MESSI Lionel|            0|
|  ANSALDI Cristian|            1|
|      BIGLIA Lucas|            1|
|       BANEGA Ever|            1|
| MASCHERANO Javier|            1|
|      DYBALA Paulo|            1|
|     AGUERO Sergio|            1|
|   HIGUAIN Gonzalo|            1|
|    DI MARIA Angel|            1|
|  LO CELSO Giovani|            1|
|  MEZA Maximiliano|            1|
|      ACUNA Marcos|            1|
|CABALLERO Wilfredo|            1|
|   MERCADO Gabriel|            1|
|  OTAMENDI Nicolas|            1|
|       ROJO Marcos|            1|
+------------------+-------------+
only showing top 20 rows



In [11]:
#4. Name of the shortest player

data.orderBy("Height", ascending=True).select("FIFA Popular Name").show(1)

+-----------------+
|FIFA Popular Name|
+-----------------+
|   YAHIA ALSHEHRI|
+-----------------+
only showing top 1 row



In [12]:
#5. Find the tallest player of all

data.orderBy("Height", ascending=False).show(1)

+-------+---+----+-----------------+----------+----------+--------------+------+------+
|   Team|  #|Pos.|FIFA Popular Name|Birth Date|Shirt Name|          Club|Height|Weight|
+-------+---+----+-----------------+----------+----------+--------------+------+------+
|Croatia| 12|  GK|    KALINIC Lovre|03.04.1990|L. KALINIĆ|KAA Gent (BEL)|   201|    96|
+-------+---+----+-----------------+----------+----------+--------------+------+------+
only showing top 1 row



In [13]:
#6. Find average height of players from Argentina

new_data = data.groupBy("Team").mean()[['Team', 'avg(Height)']].filter(data["Team"] == "Argentina")
new_data.show()

+---------+------------------+
|     Team|       avg(Height)|
+---------+------------------+
|Argentina|178.43478260869566|
+---------+------------------+



# Level 2: Spark SQL

In [14]:
mfile = 'gdrive/MyDrive/movies.csv'
rfile = 'gdrive/MyDrive/ratings.csv'

In [15]:
mdata = spark.read.csv(mfile, header=True, inferSchema=True)
rdata = spark.read.csv(rfile, header=True, inferSchema=True)

In [16]:
mdata.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [17]:
rdata.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
+------+-------+------+----------+
only showing top 5 rows



In [18]:
from pyspark.sql.functions import *

df = mdata.join(rdata, ["movieId"])
df.createOrReplaceTempView('movie')
df.printSchema() #df is my new and combined dataframe from moveis and ratings

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [28]:
#1. Find a list of the oldest released movies

newcol = spark.sql("""select substring(title, -5,4) as year from movie""")
spark.sql("ALTER TABLE movie ADD COLUMN year int").show()

AnalysisException: ignored

In [59]:
#2. How many movies are released each year



In [19]:
#3. Find the total number of movies for each rating

spark.sql('select rating, count(rating) as `Total Ratings` from movie group By rating order by rating DESC').show()

+------+-------------+
|rating|Total Ratings|
+------+-------------+
|   5.0|        15095|
|   4.5|         7723|
|   4.0|        28750|
|   3.5|        10538|
|   3.0|        20064|
|   2.5|         4449|
|   2.0|         7271|
|   1.5|         1687|
|   1.0|         3326|
|   0.5|         1101|
+------+-------------+



In [20]:
#4. Find the number of users that rated each movie

spark.sql('select title, count(distinct userId) as `Total Num Users Rated` from movie group By title order by count(userId) DESC').show(5)

+--------------------+---------------------+
|               title|Total Num Users Rated|
+--------------------+---------------------+
| Forrest Gump (1994)|                  341|
| Pulp Fiction (1994)|                  324|
|Shawshank Redempt...|                  311|
|Silence of the La...|                  304|
|Star Wars: Episod...|                  291|
+--------------------+---------------------+
only showing top 5 rows



In [21]:
#5. Whats the total rating for each movie?

spark.sql('select title, sum(rating) as `Total Rating(sum)` from movie group by title order by sum(rating) DESC').show(5)

+--------------------+-----------------+
|               title|Total Rating(sum)|
+--------------------+-----------------+
|Shawshank Redempt...|           1395.5|
| Forrest Gump (1994)|           1382.5|
| Pulp Fiction (1994)|           1379.0|
|Silence of the La...|           1258.0|
|Star Wars: Episod...|           1228.5|
+--------------------+-----------------+
only showing top 5 rows



In [22]:
#6. Whats the average rating for each movie?

spark.sql('select title, avg(rating) as `Average Rating` from movie group by title').show()

+--------------------+------------------+
|               title|    Average Rating|
+--------------------+------------------+
|Heavenly Creature...|3.6511627906976742|
|Men in Black (a.k...| 3.663157894736842|
|Snow White and th...|3.6619718309859155|
|       Psycho (1960)| 4.253246753246753|
|   Annie Hall (1977)|           4.09375|
|First Blood (Ramb...|3.4722222222222223|
|When We Were King...|            4.4375|
|In the Heat of th...| 3.909090909090909|
|    Elizabeth (1998)|3.7291666666666665|
|Man Bites Dog (C'...|              3.25|
|One False Move (1...|3.5714285714285716|
|       Quills (2000)|           3.03125|
|Before Night Fall...|3.8333333333333335|
|O Brother, Where ...|             3.875|
|Captain Corelli's...|2.3333333333333335|
|22 Jump Street (2...|             3.875|
|Night of the Livi...| 4.111111111111111|
| If Lucy Fell (1996)|               2.8|
|Mozart and the Wh...|               3.0|
|Hello, Dolly! (1969)|              4.25|
+--------------------+------------